In [1]:
import os
import pickle

In [5]:
data_path = 'data'
train_origin_data = 'data.orig'
train_annotated_data = 'Train_output'
entity_type_list = ["d", "s", "c", "i", "a", "b", "t", "p"]
all_data = ["raw_data.orig"]

In [7]:
def resolve_line(line_num, origin_line, line, entity_type_list):
    words = []
    tags = []
    for part in line.split(" "):
        part = part.strip("\n")
        if len(part) >= 2 and part[-2] == '\\' and part[-1] in entity_type_list:
            words.extend(list(part[ :-2]))
            part_tag = ["B-" + part[-1].upper()] + ["I-" + part[-1].upper()] * (len(part) - 3)
            tags.extend(part_tag)
        else:
            words.extend(list(part))
            part_tag = ["O"] * len(part)
            tags.extend(part_tag)
    # print(len(words))
    # print(len(tags))
    assert len(words) == len(tags)
    if(list(origin_line.strip()) != words):
        print("At line {} origin and annotated line don't match! ".format(line_num))
    assert words == list(origin_line.strip())
    return words, tags

In [4]:
def read_corpora(data_path, train_origin_data,train_annotated_data,entity_type_list, save_path):
    data_origin = os.path.join(data_path, train_origin_data)
    annotation = os.path.join(data_path, train_annotated_data)
    with open(data_origin) as f1:
        origin_lines = f1.readlines()
    with open(annotation) as f2:
        annotated_lines = f2.readlines()
    assert len(origin_lines) == len(annotated_lines)
    f1.close()
    f2.close()
    train_data = []
    for i in range(len(origin_lines)):
        # cache1 = origin_lines[i]
        # cache2 = annotated_lines[i]
        # print(f'length origin lines:{len(cache1)}')
        # print(f'length annotated lines:{len(cache2)}')
        # break
        words, tags = resolve_line(i+1, origin_lines[i], annotated_lines[i], entity_type_list)
        train_data.append((words, tags))
    with open(save_path, "wb") as f:
        pickle.dump(train_data, f)
    f.close()
    return train_data

In [13]:
pre_data = read_corpora(data_path, train_origin_data,train_annotated_data, entity_type_list, os.path.join(data_path, "train_data.pkl"))
# print('Data is preprocessed!')
predict = []

for i in range(len(pre_data)):
        predict.append(pre_data[i][1])



groud_truth = 'Ground_truth'
groud_truth = os.path.join(data_path, groud_truth)
with open(groud_truth) as f:
        true_label = f.readlines()
print('Total true labels',len(true_label))
print('Total pre labels',len(pre_data))
print('Total pre labels',len(predict))




Total true labels 197
Total pre labels 197
Total pre labels 197


In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.preprocessing import MultiLabelBinarizer  

true_label = MultiLabelBinarizer().fit_transform(true_label)
predict = MultiLabelBinarizer().fit_transform(predict)

acc = accuracy_score(true_label,predict)
precision = precision_score(true_label,predict,average='micro')
recall = recall_score(true_label,predict,average='micro')
f1 = f1_score(true_label,predict,average='micro')


print(f'acc:{acc}')
print(f'precision:{precision}')
print(f'recall:{recall}')
print(f'f1:{f1}')

acc:0.9086294416243654
precision:0.952755905511811
recall:1.0
f1:0.9758064516129031
